In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import alexnet
from torchvision import datasets
from collections import OrderedDict

from fftKAN import NaiveFourierKANLayer

# Import the CIFAR-10 dataset
datasets.CIFAR10(root='./data', train=True, download=True)

# Create the AlexNet model
model = alexnet()

features = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(64, 192, kernel_size=5, padding=2),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(192, 384, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(384, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(256, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
)
avgpool = nn.AdaptiveAvgPool2d((6, 6))
classifier = nn.Sequential(
    nn.Dropout(),
    NaiveFourierKANLayer(256 * 6 * 6, 1000, gridsize=50),
    nn.Dropout(),
    NaiveFourierKANLayer(1000, 1000, gridsize=100),
    NaiveFourierKANLayer(1000, 1000, gridsize=100),
)
handmade_model = nn.Sequential(OrderedDict([
    ('features', features),
    ('avgpool', avgpool),
    ('classifier', classifier)
]))

Files already downloaded and verified
